In [1]:
"""
Product Matcher v4 - GPU Optimized (RTX 3050 Ready)
===================================================
- Исправлены все deprecated параметры
- Оптимизация для 4GB VRAM
- FAISS на CPU (GPU версия опциональна)
- Encoder + Reranker на GPU
"""

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
import re
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional
import warnings
import gc
import os
from pathlib import Path
import torch

warnings.filterwarnings('ignore')


def check_gpu_status():
    """Проверка статуса GPU"""
    print("=" * 60)
    print("🔍 ПРОВЕРКА GPU")
    print("=" * 60)
    
    print(f"\n📦 PyTorch version: {torch.__version__}")
    print(f"🔧 CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"🎮 CUDA version: {torch.version.cuda}")
        print(f"📊 GPU count: {torch.cuda.device_count()}")
        
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            print(f"\n   GPU {i}: {props.name}")
            print(f"   Memory: {props.total_memory / 1e9:.1f} GB")
            print(f"   Compute capability: {props.major}.{props.minor}")
        
        print(f"\n💾 Текущее использование VRAM:")
        print(f"   Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        print(f"   Cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
    else:
        print("⚠️  GPU недоступен! Будет использован CPU.")
    
    print(f"\n📦 FAISS GPU support: {hasattr(faiss, 'StandardGpuResources')}")
    print("=" * 60)
    
    return torch.cuda.is_available()


class ProductMatcherGPU:
    """
    GPU-оптимизированный матчер товаров:
    1. Retrieval: BGE-M3 (GPU) + FAISS → топ-K кандидатов
    2. Reranking: BGE-reranker-v2-m3 (GPU) → лучший кандидат
    """
    
    _encoder_instance = None
    _reranker_instance = None
    
    def __init__(self, 
                 encoder_model: str = 'BAAI/bge-m3',
                 reranker_model: str = 'BAAI/bge-reranker-v2-m3',
                 cache_dir: str = './cache',
                 use_fp16: bool = True,
                 gpu_id: int = 0,
                 force_gpu: bool = False):
        """
        Args:
            encoder_model: Модель для эмбеддингов
            reranker_model: Модель для переранжирования
            cache_dir: Директория для кеша
            use_fp16: Использовать float16 для экономии VRAM
            gpu_id: ID GPU для использования
            force_gpu: Принудительно требовать GPU
        """
        
        self.gpu_available = torch.cuda.is_available()
        self.gpu_id = gpu_id
        
        if force_gpu and not self.gpu_available:
            raise RuntimeError("❌ GPU не найден!")
        
        # Инициализация атрибутов
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.faiss_index: Optional[faiss.Index] = None
        self.gpu_resources = None
        self.eva_texts_clean: Optional[List[str]] = None
        self.eva_df: Optional[pd.DataFrame] = None
        
        # Устройство
        if self.gpu_available:
            self.device = f'cuda:{gpu_id}'
            torch.cuda.set_device(gpu_id)
        else:
            self.device = 'cpu'
        
        print("=" * 60)
        print("🚀 Инициализация ProductMatcherGPU")
        print("=" * 60)
        print(f"🖥️  Device: {self.device}")
        
        if self.gpu_available:
            props = torch.cuda.get_device_properties(gpu_id)
            self.vram_total = props.total_memory / 1e9
            print(f"🎮 GPU: {props.name}")
            print(f"💾 VRAM: {self.vram_total:.1f} GB")
            
            # FAISS-GPU (опционально)
            self.faiss_gpu_available = hasattr(faiss, 'StandardGpuResources')
            if self.faiss_gpu_available:
                try:
                    self.gpu_resources = faiss.StandardGpuResources()
                    self.gpu_resources.setTempMemory(256 * 1024 * 1024)  # 256MB
                    print(f"🔧 FAISS-GPU: ✅ Доступен")
                except:
                    self.faiss_gpu_available = False
                    print(f"🔧 FAISS-GPU: ❌ Ошибка инициализации")
            else:
                print(f"🔧 FAISS-GPU: ❌ Не установлен (используется CPU)")
        else:
            self.vram_total = 0
            self.faiss_gpu_available = False
        
        # Dtype
        if use_fp16 and self.gpu_available:
            self.model_dtype = torch.float16
            print("⚡ Precision: FP16")
        else:
            self.model_dtype = torch.float32
            print("📊 Precision: FP32")
        
        # Загрузка моделей
        self._load_encoder(encoder_model)
        self._load_reranker(reranker_model)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        print("\n✅ Инициализация завершена!")
        print("=" * 60)
    
    def _load_encoder(self, model_name: str):
        """Загрузка encoder на GPU"""
        if ProductMatcherGPU._encoder_instance is None:
            print(f"\n📥 Загрузка encoder: {model_name}")
            
            try:
                # Современный способ с dtype
                ProductMatcherGPU._encoder_instance = SentenceTransformer(
                    model_name,
                    device=self.device,
                    model_kwargs={
                        'torch_dtype': self.model_dtype,
                    }
                )
            except Exception as e1:
                print(f"   ⚠️ Попытка 1 не удалась: {e1}")
                try:
                    # Fallback без kwargs
                    ProductMatcherGPU._encoder_instance = SentenceTransformer(
                        model_name,
                        device=self.device
                    )
                    if self.model_dtype == torch.float16 and self.gpu_available:
                        ProductMatcherGPU._encoder_instance.half()
                except Exception as e2:
                    print(f"   ❌ Ошибка загрузки: {e2}")
                    raise
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._encoder_instance.parameters()).device
                    print(f"   ✅ Encoder на: {dev}")
                except:
                    pass
        else:
            print("✅ Encoder уже загружен")
        
        self.encoder = ProductMatcherGPU._encoder_instance
    
    def _load_reranker(self, model_name: str):
        """Загрузка reranker на GPU"""
        if ProductMatcherGPU._reranker_instance is None:
            print(f"\n📥 Загрузка reranker: {model_name}")
            
            try:
                # Современный способ с model_kwargs
                ProductMatcherGPU._reranker_instance = CrossEncoder(
                    model_name,
                    max_length=512,
                    device=self.device,
                    model_kwargs={'torch_dtype': self.model_dtype}
                )
            except Exception as e1:
                print(f"   ⚠️ Попытка 1 не удалась: {e1}")
                try:
                    # Fallback
                    ProductMatcherGPU._reranker_instance = CrossEncoder(
                        model_name,
                        max_length=512,
                        device=self.device
                    )
                    if self.model_dtype == torch.float16 and self.gpu_available:
                        ProductMatcherGPU._reranker_instance.model.half()
                except Exception as e2:
                    print(f"   ❌ Ошибка загрузки: {e2}")
                    raise
            
            if self.gpu_available:
                try:
                    dev = next(ProductMatcherGPU._reranker_instance.model.parameters()).device
                    print(f"   ✅ Reranker на: {dev}")
                except:
                    pass
        else:
            print("✅ Reranker уже загружен")
        
        self.reranker = ProductMatcherGPU._reranker_instance
    
    def _print_gpu_memory(self):
        """Вывод использования GPU"""
        if self.gpu_available:
            allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
            reserved = torch.cuda.memory_reserved(self.gpu_id) / 1e9
            print(f"\n💾 GPU Memory: {allocated:.2f} / {self.vram_total:.1f} GB (reserved: {reserved:.2f} GB)")
    
    def _get_free_vram(self) -> float:
        """Получить свободную VRAM в GB"""
        if not self.gpu_available:
            return 0
        allocated = torch.cuda.memory_allocated(self.gpu_id) / 1e9
        return self.vram_total - allocated
    
    @classmethod
    def clear_models(cls):
        """Очистка моделей и GPU памяти"""
        print("🧹 Очистка моделей...")
        cls._encoder_instance = None
        cls._reranker_instance = None
        gc.collect()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
            print(f"   GPU memory: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
        
        print("✅ Очищено")
    
    # =========================================================================
    # Очистка текста
    # =========================================================================
    
    def clean_text(self, text: str) -> str:
        """Нормализация текста"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower().strip()
        
        # Единицы измерения
        replacements = [
            (r'(\d+)\s*мл\b', r'\1мл'),
            (r'(\d+)\s*ml\b', r'\1мл'),
            (r'(\d+)\s*г\b', r'\1г'),
            (r'(\d+)\s*g\b', r'\1г'),
            (r'(\d+)\s*кг\b', r'\1кг'),
            (r'(\d+)\s*kg\b', r'\1кг'),
            (r'(\d+)\s*л\b', r'\1л'),
            (r'(\d+)\s*l\b', r'\1л'),
            (r'(\d+)\s*шт\b', r'\1шт'),
        ]
        
        for pattern, repl in replacements:
            text = re.sub(pattern, repl, text)
        
        # Очистка
        text = re.sub(r'[^\w\s\-\.,/]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        
        return text.strip()
    
    def clean_texts_batch(self, texts: List[str], desc: str = "Очистка") -> List[str]:
        """Батч-очистка текстов"""
        return [self.clean_text(t) for t in tqdm(texts, desc=desc)]
    
    # =========================================================================
    # Эмбеддинги
    # =========================================================================
    
    def create_embeddings(self, 
                          texts: List[str], 
                          batch_size: int = 8,
                          show_progress: bool = True) -> np.ndarray:
        """Создание эмбеддингов (GPU)"""
        
        # Автоподбор batch_size для RTX 3050
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.5:
                batch_size = min(batch_size, 4)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 3.0:
                batch_size = min(batch_size, 16)
            
            print(f"   Batch size: {batch_size} (free VRAM: {free_vram:.1f} GB)")
        
        embeddings = self.encoder.encode(
            texts,
            batch_size=batch_size,
            show_progress_bar=show_progress,
            normalize_embeddings=True,
            convert_to_numpy=True,
            device=self.device
        )
        
        return embeddings.astype('float32')
    
    # =========================================================================
    # FAISS
    # =========================================================================
    
    def create_faiss_index(self, embeddings: np.ndarray) -> faiss.Index:
        """Создание FAISS индекса"""
        dimension = embeddings.shape[1]
        
        # CPU индекс (надёжнее)
        index = faiss.IndexFlatIP(dimension)
        index.add(embeddings)
        
        # GPU если доступен и достаточно памяти
        if self.faiss_gpu_available and self.gpu_resources and self._get_free_vram() > 0.5:
            try:
                gpu_index = faiss.index_cpu_to_gpu(self.gpu_resources, self.gpu_id, index)
                print(f"   ✅ FAISS на GPU")
                return gpu_index
            except Exception as e:
                print(f"   ⚠️ FAISS на GPU не удалось: {e}")
        
        print(f"   📊 FAISS на CPU")
        return index
    
    def _index_to_cpu(self, index: faiss.Index) -> faiss.Index:
        """Перенос индекса на CPU"""
        if hasattr(faiss, 'index_gpu_to_cpu'):
            try:
                return faiss.index_gpu_to_cpu(index)
            except:
                pass
        return index
    
    # =========================================================================
    # Кеширование
    # =========================================================================
    
    def _get_cache_path(self, name: str) -> Path:
        return self.cache_dir / name
    
    def save_index(self, 
                   df: pd.DataFrame,
                   text_column: str,
                   cache_name: str = 'products'):
        """Создание и сохранение индекса"""
        print("\n" + "=" * 60)
        print(f"💾 Создание индекса ({len(df)} товаров)")
        print("=" * 60)
        
        # Очистка
        print("\n1️⃣ Очистка текстов...")
        texts_clean = self.clean_texts_batch(df[text_column].tolist())
        
        # Эмбеддинги
        print("\n2️⃣ Создание эмбеддингов...")
        embeddings = self.create_embeddings(texts_clean)
        
        # Индекс
        print("\n3️⃣ Создание FAISS индекса...")
        index = self.create_faiss_index(embeddings)
        
        # Сохранение
        print("\n4️⃣ Сохранение...")
        cpu_index = self._index_to_cpu(index)
        
        np.save(self._get_cache_path(f'{cache_name}_embeddings.npy'), embeddings)
        pd.DataFrame({'text_clean': texts_clean}).to_parquet(
            self._get_cache_path(f'{cache_name}_texts.parquet')
        )
        faiss.write_index(cpu_index, str(self._get_cache_path(f'{cache_name}_index.faiss')))
        df.to_parquet(self._get_cache_path(f'{cache_name}_products.parquet'))
        
        print(f"✅ Сохранено в {self.cache_dir}/")
        
        self.faiss_index = index
        self.eva_texts_clean = texts_clean
        self.eva_df = df
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return index, texts_clean
    
    def load_index(self, cache_name: str = 'products') -> bool:
        """Загрузка индекса из кеша"""
        paths = [
            self._get_cache_path(f'{cache_name}_embeddings.npy'),
            self._get_cache_path(f'{cache_name}_texts.parquet'),
            self._get_cache_path(f'{cache_name}_index.faiss'),
            self._get_cache_path(f'{cache_name}_products.parquet'),
        ]
        
        if not all(p.exists() for p in paths):
            print("⚠️ Кеш не найден")
            return False
        
        print("\n📂 Загрузка из кеша...")
        
        self.faiss_index = faiss.read_index(str(paths[2]))
        self.eva_texts_clean = pd.read_parquet(paths[1])['text_clean'].tolist()
        self.eva_df = pd.read_parquet(paths[3])
        
        print(f"✅ Загружено {self.faiss_index.ntotal} товаров")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return True
    
    # =========================================================================
    # Поиск и Reranking
    # =========================================================================
    
    def search_candidates(self, 
                          query_embeddings: np.ndarray,
                          top_k: int = 10) -> Tuple[np.ndarray, np.ndarray]:
        """Поиск кандидатов"""
        scores, indices = self.faiss_index.search(query_embeddings.astype('float32'), top_k)
        return indices, scores
    
    def rerank_batch(self,
                     query_texts: List[str],
                     candidates_indices: np.ndarray,
                     batch_size: int = 16) -> List[Tuple[int, float, int]]:
        """Переранжирование кандидатов"""
        
        # Собираем пары
        all_pairs = []
        pair_info = []
        
        for q_idx, (query, cand_indices) in enumerate(zip(query_texts, candidates_indices)):
            for pos, idx in enumerate(cand_indices):
                if idx >= 0:
                    all_pairs.append([query, self.eva_texts_clean[idx]])
                    pair_info.append((q_idx, pos, idx))
        
        # Автоподбор batch_size
        if self.gpu_available:
            free_vram = self._get_free_vram()
            if free_vram < 1.0:
                batch_size = min(batch_size, 8)
            elif free_vram < 1.5:
                batch_size = min(batch_size, 16)
            elif free_vram < 2.0:
                batch_size = min(batch_size, 32)
        
        print(f"   Reranking {len(all_pairs)} пар (batch={batch_size})...")
        
        # Predict
        all_scores = self.reranker.predict(
            all_pairs, 
            batch_size=batch_size,
            show_progress_bar=True
        )
        
        # Группируем
        query_results = {}
        for (q_idx, pos, idx), score in zip(pair_info, all_scores):
            if q_idx not in query_results:
                query_results[q_idx] = []
            query_results[q_idx].append((idx, float(score), pos))
        
        # Лучший для каждого
        results = []
        for q_idx in range(len(query_texts)):
            if q_idx in query_results:
                best = max(query_results[q_idx], key=lambda x: x[1])
                results.append(best)
            else:
                results.append((-1, 0.0, -1))
        
        return results
    
    # =========================================================================
    # Основной метод
    # =========================================================================
    
    def match_products(self,
                       competitor_df: pd.DataFrame,
                       competitor_col: str = 'name',
                       our_df: Optional[pd.DataFrame] = None,
                       our_col: str = 'name',
                       top_k: int = 10,
                       threshold: float = 0.5,
                       encoder_batch: int = 8,
                       reranker_batch: int = 16,
                       use_cache: bool = True,
                       cache_name: str = 'products') -> pd.DataFrame:
        """
        Сопоставление товаров
        
        Args:
            competitor_df: DataFrame конкурента
            competitor_col: Колонка с названием товара
            our_df: Наш DataFrame (или None если есть кеш)
            our_col: Колонка с названием товара
            top_k: Количество кандидатов для reranking
            threshold: Порог уверенности
            encoder_batch: Batch size для encoder
            reranker_batch: Batch size для reranker
            use_cache: Использовать кеш
            cache_name: Имя кеша
        """
        print("\n" + "=" * 70)
        print("🚀 СОПОСТАВЛЕНИЕ ТОВАРОВ")
        print("=" * 70)
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        # 1. Индекс наших товаров
        if self.faiss_index is None:
            if use_cache and self.load_index(cache_name):
                pass
            elif our_df is not None:
                self.save_index(our_df, our_col, cache_name)
            else:
                raise ValueError("Укажите our_df или используйте кеш")
        
        print(f"\n📦 Наши товары: {self.faiss_index.ntotal}")
        print(f"📦 Конкурент: {len(competitor_df)}")
        
        # 2. Обработка конкурента
        print("\n" + "-" * 50)
        print("📝 Этап 1: Подготовка товаров конкурента")
        print("-" * 50)
        
        comp_texts = competitor_df[competitor_col].tolist()
        comp_texts_clean = self.clean_texts_batch(comp_texts, "Очистка")
        
        print("\nСоздание эмбеддингов...")
        comp_embeddings = self.create_embeddings(comp_texts_clean, batch_size=encoder_batch)
        
        # 3. Retrieval
        print("\n" + "-" * 50)
        print(f"🔍 Этап 2: Retrieval (топ-{top_k})")
        print("-" * 50)
        
        cand_indices, cand_scores = self.search_candidates(comp_embeddings, top_k)
        print(f"   ✅ Найдено кандидатов для {len(cand_indices)} товаров")
        
        # 4. Reranking
        print("\n" + "-" * 50)
        print("🎯 Этап 3: Reranking")
        print("-" * 50)
        
        rerank_results = self.rerank_batch(comp_texts_clean, cand_indices, batch_size=reranker_batch)
        
        # 5. Результаты
        print("\n" + "-" * 50)
        print("📊 Этап 4: Формирование результатов")
        print("-" * 50)
        
        # Определяем колонку
        if our_col in self.eva_df.columns:
            eva_col = our_col
        else:
            eva_col = self.eva_df.columns[1]
        
        results = []
        for comp_idx, (eva_idx, score, rank) in enumerate(rerank_results):
            
            status = "matched" if score >= threshold else "low_confidence"
            
            results.append({
                'competitor_index': comp_idx,
                'competitor_product': competitor_df.iloc[comp_idx][competitor_col],
                'our_index': eva_idx if status == "matched" else None,
                'our_product': self.eva_df.iloc[eva_idx][eva_col] if eva_idx >= 0 else None,
                'retrieval_score': float(cand_scores[comp_idx][rank]) if rank >= 0 else 0.0,
                'rerank_score': score,
                'retrieval_rank': rank + 1 if rank >= 0 else -1,
                'match_status': status
            })
        
        result_df = pd.DataFrame(results)
        
        # Статистика
        matched = len(result_df[result_df['match_status'] == 'matched'])
        low_conf = len(result_df[result_df['match_status'] == 'low_confidence'])
        
        print(f"\n✅ Готово!")
        print(f"   Сопоставлено: {matched} ({matched/len(result_df)*100:.1f}%)")
        print(f"   Низкая уверенность: {low_conf} ({low_conf/len(result_df)*100:.1f}%)")
        
        if self.gpu_available:
            self._print_gpu_memory()
        
        return result_df
    
    def analyze_results(self, df: pd.DataFrame, show_examples: int = 5) -> Dict:
        """Анализ результатов"""
        print("\n" + "=" * 70)
        print("📊 АНАЛИЗ РЕЗУЛЬТАТОВ")
        print("=" * 70)
        
        total = len(df)
        matched = df[df['match_status'] == 'matched']
        low_conf = df[df['match_status'] == 'low_confidence']
        
        print(f"\n📈 Статистика:")
        print(f"   Всего: {total}")
        print(f"   ✅ Сопоставлено: {len(matched)} ({len(matched)/total*100:.1f}%)")
        print(f"   ⚠️  Низкая уверенность: {len(low_conf)} ({len(low_conf)/total*100:.1f}%)")
        
        print(f"\n📉 Rerank score:")
        print(f"   Среднее: {df['rerank_score'].mean():.4f}")
        print(f"   Медиана: {df['rerank_score'].median():.4f}")
        print(f"   Мин/Макс: {df['rerank_score'].min():.4f} / {df['rerank_score'].max():.4f}")
        
        print(f"\n📊 Распределение:")
        for t in [0.9, 0.7, 0.5, 0.3]:
            count = len(df[df['rerank_score'] >= t])
            print(f"   >= {t}: {count} ({count/total*100:.1f}%)")
        
        if show_examples > 0 and len(matched) > 0:
            print(f"\n📋 Топ-{show_examples} лучших:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(matched.nlargest(show_examples, 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Конкурент: {row['competitor_product'][:65]}")
                print(f"   Наш:       {str(row['our_product'])[:65]}")
        
        if show_examples > 0 and len(low_conf) > 0:
            print(f"\n⚠️  Худшие {min(3, len(low_conf))}:")
            print("-" * 70)
            
            for i, (_, row) in enumerate(low_conf.nsmallest(min(3, len(low_conf)), 'rerank_score').iterrows(), 1):
                print(f"\n{i}. Score: {row['rerank_score']:.4f}")
                print(f"   Конкурент: {row['competitor_product'][:65]}")
                print(f"   Лучший:    {str(row['our_product'])[:65]}")
        
        return {
            'total': total,
            'matched': len(matched),
            'low_confidence': len(low_conf),
            'avg_score': df['rerank_score'].mean(),
            'median_score': df['rerank_score'].median()
        }


# =============================================================================
# БЫСТРЫЙ ЗАПУСК
# =============================================================================

def quick_match(competitor_csv: str,
                our_csv: str,
                output_csv: str = 'matches.csv',
                competitor_col: str = 'name',
                our_col: str = 'name',
                threshold: float = 0.5,
                top_k: int = 5) -> pd.DataFrame:
    """
    Быстрый матчинг из CSV файлов
    
    Пример:
        results = quick_match('competitor.csv', 'our_products.csv')
    """
    print("📂 Загрузка данных...")
    competitor_df = pd.read_csv(competitor_csv)
    our_df = pd.read_csv(our_csv)
    
    print(f"   Конкурент: {len(competitor_df)} товаров")
    print(f"   Наши: {len(our_df)} товаров")
    
    # Матчер
    matcher = ProductMatcherGPU(use_fp16=True, force_gpu=False)
    
    # Матчинг (параметры для 4GB VRAM)
    results = matcher.match_products(
        competitor_df,
        competitor_col=competitor_col,
        our_df=our_df,
        our_col=our_col,
        top_k=top_k,
        threshold=threshold,
        encoder_batch=8,
        reranker_batch=16,
        use_cache=True
    )
    
    # Анализ
    matcher.analyze_results(results)
    
    # Сохранение
    results.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"\n💾 Сохранено: {output_csv}")
    
    return results


if __name__ == "__main__":
    # 1. Проверяем статус GPU перед запуском
    check_gpu_status()

    # 2. Читаем файлы
    print("\n📂 ЗАГРУЗКА ДАННЫХ:")
    try:
        # Если у вас разделитель точка с запятой, добавьте аргумент: sep=';'
        df_competitor = pd.read_csv("competitor_products.csv") 
        df_our = pd.read_csv("our_products.csv")
    except FileNotFoundError:
        print("❌ Ошибка: Файлы не найдены.")
        print("   Убедитесь, что 'competitor_products.csv' и 'our_products.csv' лежат рядом со скриптом.")
        exit()

    # 3. Анализ колонок (помогает понять, что вписывать в настройки)
    print("\n🔍 АНАЛИЗ КОЛОНОК:")
    print(f"Файл конкурента (df_competitor): {df_competitor.columns.tolist()}")
    print(f"Наш файл (df_our):               {df_our.columns.tolist()}")
    print("-" * 60)

    # =========================================================================
    # ⚙️ НАСТРОЙКИ (ОТРЕДАКТИРУЙТЕ ЭТОТ БЛОК)
    # =========================================================================
    
    # ❗ Впишите сюда точные названия колонок с наименованиями товаров
    COMPETITOR_COL_NAME = 'name'   # Название колонки в competitor_products.csv
    OUR_COL_NAME = 'name'          # Название колонки в our_products.csv

    # Настройки производительности
    USE_FP16 = True                # True для RTX 3050 (экономит память)
    BATCH_SIZE_ENCODER = 8         # Для 4GB VRAM лучше 4 или 8
    BATCH_SIZE_RERANKER = 16       # Для 4GB VRAM лучше 16
    TOP_K_CANDIDATES = 5           # Сколько кандидатов перепроверять реранкером

    # =========================================================================

    # 4. Инициализация класса
    # Можно указать cache_dir, чтобы при повторном запуске не считать эмбеддинги наших товаров заново
    matcher = ProductMatcherGPU(
        use_fp16=USE_FP16, 
        cache_dir='./cache_products'
    )
    
    # 5. Запуск сопоставления
    # Обратите внимание: сигнатура метода match_products изменилась в новой версии алгоритма
    try:
        results = matcher.match_products(
            competitor_df=df_competitor,
            competitor_col=COMPETITOR_COL_NAME,
            our_df=df_our,
            our_col=OUR_COL_NAME,
            top_k=TOP_K_CANDIDATES,
            threshold=0.5,             # Порог уверенности (0.0 - 1.0)
            encoder_batch=BATCH_SIZE_ENCODER,
            reranker_batch=BATCH_SIZE_RERANKER,
            use_cache=True             # Используем кеш для ускорения
        )

        # 6. Анализ и сохранение
        matcher.analyze_results(results, show_examples=5)
        
        output_file = 'results_gpu_matched.csv'
        results.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n💾 Результаты сохранены в файл: {output_file}")

    except KeyError as e:
        print(f"\n❌ ОШИБКА КОЛОНОК: Не найдена колонка {e}")
        print("   Проверьте блок 'НАСТРОЙКИ' и сравните названия с выводом 'АНАЛИЗ КОЛОНОК'.")
    except Exception as e:
        print(f"\n❌ Произошла ошибка при выполнении: {e}")
    finally:
        # Очистка памяти GPU в конце работы
        ProductMatcherGPU.clear_models()


🔍 ПРОВЕРКА GPU

📦 PyTorch version: 2.6.0+cu124
🔧 CUDA available: True
🎮 CUDA version: 12.4
📊 GPU count: 1

   GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
   Memory: 4.3 GB
   Compute capability: 8.6

💾 Текущее использование VRAM:
   Allocated: 0.00 GB
   Cached: 0.00 GB

📦 FAISS GPU support: False

📂 ЗАГРУЗКА ДАННЫХ:

🔍 АНАЛИЗ КОЛОНОК:
Файл конкурента (df_competitor): ['id', 'name']
Наш файл (df_our):               ['id', 'name']
------------------------------------------------------------
🚀 Инициализация ProductMatcherGPU
🖥️  Device: cuda:0
🎮 GPU: NVIDIA GeForce RTX 3050 Laptop GPU
💾 VRAM: 4.3 GB
🔧 FAISS-GPU: ❌ Не установлен (используется CPU)
⚡ Precision: FP16

📥 Загрузка encoder: BAAI/bge-m3


`torch_dtype` is deprecated! Use `dtype` instead!


   ✅ Encoder на: cuda:0

📥 Загрузка reranker: BAAI/bge-reranker-v2-m3
   ✅ Reranker на: cuda:0

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)

✅ Инициализация завершена!

🚀 СОПОСТАВЛЕНИЕ ТОВАРОВ

💾 GPU Memory: 2.27 / 4.3 GB (reserved: 2.28 GB)
⚠️ Кеш не найден

💾 Создание индекса (500 товаров)

1️⃣ Очистка текстов...


Очистка: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 61116.51it/s]


2️⃣ Создание эмбеддингов...
   Batch size: 8 (free VRAM: 2.0 GB)


...


Создание эмбеддингов...
   Batch size: 8 (free VRAM: 2.0 GB)


Batches:   0%|          | 0/63 [00:00<?, ?it/s]


--------------------------------------------------
🔍 Этап 2: Retrieval (топ-5)
--------------------------------------------------
   ✅ Найдено кандидатов для 500 товаров

--------------------------------------------------
🎯 Этап 3: Reranking
--------------------------------------------------
   Reranking 2500 пар (batch=16)...


Batches:   0%|          | 0/157 [00:00<?, ?it/s]


--------------------------------------------------
📊 Этап 4: Формирование результатов
--------------------------------------------------

✅ Готово!
   Сопоставлено: 500 (100.0%)
   Низкая уверенность: 0 (0.0%)

💾 GPU Memory: 2.28 / 4.3 GB (reserved: 2.33 GB)

📊 АНАЛИЗ РЕЗУЛЬТАТОВ

📈 Статистика:
   Всего: 500
   ✅ Сопоставлено: 500 (100.0%)
   ⚠️  Низкая уверенность: 0 (0.0%)

📉 Rerank score:
   Среднее: 0.9911
   Медиана: 0.9995
   Мин/Макс: 0.7734 / 1.0000

📊 Распределение:
   >= 0.9: 489 (97.8%)
   >= 0.7: 500 (100.0%)
   >= 0.5: 500 (100.0%)
   >= 0.3: 500 (100.0%)

📋 Топ-5 лучших:
----------------------------------------------------------------------

1. Score: 1.0000
   Конкурент: Pantene ProV Шампунь объем 250мл
   Наш:       Pantene Pro-V Shamp объем 250мл

2. Score: 1.0000
   Конкурент: L'Oreal Elseve ШАМПУНЬ против выпадения 400мл
   Наш:       Shamp Эльсев - против выпадения / 400мл

3. Score: 1.0000
   Конкурент: ШАМПУНЬ Elseve для окрашенных волос 250мл
   Наш:       для о